# Epperlein-Short test 

This notebook uses the prebuilt Epperlein-Short run wrapper generator in es_test.py to perform the standard Epperlein-Short test.

This test corresponds to Section 5.2.3. in the ReMKiT1D code paper.

In [7]:
import numpy as np
import xarray as xr
import holoviews as hv
import matplotlib.pyplot as plt
import matplotlib as mpl
from holoviews import opts
import panel as pn
from es_test import esTestGenerator
import pickle

import sys
sys.path.append('../')
import RMK_support.IO_support as io
import RMK_support.dashboard_support as ds

### Set up problem parameters

In [8]:
kLambda = 1e-2 # Braginskii k * mfp - To reproduce points in Figure 12 in the paper use values from np.geomspace(0.5e-2,2,8)
k = kLambda/(3*np.sqrt(np.pi)/(4*np.sqrt(2)))
Nx = 128 # number of spatal grids
dx = 2*np.pi/(k*Nx) 
dt = 0.05 # time step in e-i collional times
ionZ = 1.0 # ion charge
L = Nx*dx # total domain length
Nt=300 # number of timesteps
lmax=1 # highest resolved harmonic - change to reproduce points in Figure 12 in the paper

### Initialize wrapper using prebuilt script

In [9]:
rk = esTestGenerator(dx=dx,
                     Nx=Nx,
                     lmax=lmax,
                     ionZ=ionZ,
                     mpiProcsX=16,
                     mpiProcsH=1,
                     hdf5Filepath="./RMKOutput/RMK_ES_test/",
                     initialTimestep=dt,
                     )

In [10]:
rk.setPETScOptions(cliOpts="-pc_type bjacobi -sub_pc_factor_shift_type nonzero",kspSolverType="gmres")

### Set timesteps and output config file

In [11]:
rk.setFixedNumTimesteps(Nt)
rk.setFixedStepOutput(Nt/30)

In [12]:
rk.writeConfigFile()

### Load and analyze data

In [ ]:
numFiles=30

In [ ]:
loadpath = rk.hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]


In [ ]:
loadedData = io.loadFromHDF5(rk.varCont, filepaths=loadFilenames, varsToIgnore=["zeroVar","ni"])
loadedData


In [ ]:
hv.extension('matplotlib')
%matplotlib inline
plt.rcParams['figure.dpi'] = 150
hv.output(size=80, dpi=150)


### Compare heat flux with Braginskii value visually

In [ ]:
pn.extension(comms="vscode")  # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData,rk.grid)

dashboard.fluidMultiComparison(["qT","q_dual"]).opts(ylabel="q (normalized units)")


### Calculate heat flux suppression using two methods

In [ ]:
maxCoords = [np.argmax(loadedData["T"].data[t,:]) for t in range(numFiles+1)]

In [ ]:
def qRatio(loadedData:xr.Dataset,maxCoord:int) -> float:
    """Return ratio of heat flux to Braginskii value at given position

    Args:
        loadedData (xr.Dataset): xarray dataset with run results
        maxCoord (int): Position of temperature perturbation maximum

    Returns:
        float: Heat flux ratio that can be interpreted as a conductivity ratio
    """
    return 1 - abs(loadedData["qT"][:,maxCoord]-loadedData["q"][:,maxCoord])/abs(loadedData["qT"][:,maxCoord])

In [ ]:
def rhoRatio(loadedData:xr.Dataset,maxCoord:int,ionZ:float,k:float)->float:
    """Return ratio of temperature decay rate to expected Braginskii value at given position

    Args:
        loadedData (xr.Dataset): xarray dataset with run results
        maxCoord (int): Position of temperature perturbation maximum
        ionZ (float): Ion charge
        k (float): Perturbation wave number

    Returns:
        float: Temperature decay ratio that can be interpreted as a conductivity ratio
    """
    kappaOne = 3.16 # Thermal conductivity for Z=1
    kappaB = (ionZ+0.24)/(0.24*ionZ+1)*kappaOne # Thermal conductivity scaling based off of original Epperlein paper
    rhoB=k**2*np.sqrt(np.pi)/4 * kappaB # Expected Braginskii value
    Nt = len(loadedData.coords["time"])
    deltaT = loadedData["T"].data[:,maxCoord] - 1
    dAmpT = np.log(deltaT[1:]/deltaT[:Nt-1])
    dt = loadedData.coords["time"].data[1:] - loadedData.coords["time"].data[:Nt-1]
    return - dAmpT/(rhoB*dt)
    

In [ ]:
rRatio=rhoRatio(loadedData,maxCoord=maxCoords[1],ionZ=ionZ,k=k)
rRatio

In [ ]:
qRatio = qRatio(loadedData,maxCoord=maxCoords[1])
qRatio

In [ ]:
plt.plot(rRatio,label="$\\rho$ ratio")
plt.plot(qRatio,label="q ratio")
plt.legend()
